# Prep

## Dependencies

In [ ]:
!pip install torchsummary

In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchsummary
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset, ConcatDataset, WeightedRandomSampler
from torchsummary import summary
from torchvision.io import read_image
from collections import Counter

import numpy as np
from PIL import ImageStat

## Hyperparameter Settings -- variables to be tweaked

In [ ]:
# Model architecture parameter
# Available architectures
# resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110'
# 'vgg19', 'alexnet', 
# 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'
arch = 'resnet20'
resize = [32,32]

# Learning parameters
weight_decay = 1e-4            # (1e-4 - float?) Weight decay
momentum = 0.9                  # (0.9 - dec) Learning momentum
lr = 0.01                        # (0.1 - dec) Learning rate
start_epoch = 0                 # (0 - int) First epoch value
epochs = 50                    # (200 - int) How many epochs
batch_size_train = 64           # (64 - int) Size of batches
batch_size_val = 64             # (64 - int) Size of load batches
grayscale = False

# Performance optimizations
half = False                    # (0 - bool) Use half-precision (16-bit)
num_workers = 2                 # Set to 2 for Kaggle

# Trained model parameters
evaluate = False                # (0 - bool) evaluate model
pretrained = False              # (0 - bool) evaluate pretrained model
print_freq = 999                # (999 - int) print frequency

# Input/Output parameters
save_every = 10                # (10 - int) save checkpoint intervals

# ('save_temp' - string) The directory used to save the trained models
save_dir = 'save_temp'

do_normalize = False
if grayscale:
    normalize_gray = transforms.Normalize(mean=0.6587, std=0.2029)
else:
    normalize_rgb = transforms.Normalize(mean=[0.7914, 0.6591, 0.4151], std=[0.1208, 0.1548, 0.1494])

# Resnet Architecture for small image datasets (i.e. CIFAR-10, Sundanese Lontar Character Set)

Contains resnet20, resnet32, resnet44, resnet56, resnet110, resnet1202*

In [ ]:
__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 
           'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 
           'vgg19', 'alexnet']


def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)


class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                '''
                For CIFAR10 ResNet paper uses option A.
                '''
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_planes, self.expansion * planes,
                              kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=28):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)
        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])

def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])

def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])

def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])

def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])

# predefined architectures from torchvision

def vgg19():
    return models.vgg19()

def alexnet():
    return models.alexnet()

def resnet18():
    return models.resnet18()

def resnet34():
    return models.resnet34()

def resnet50():
    return models.resnet50()

def resnet101():
    return models.resnet101()

def resnet152():
    return models.resnet152()

def test(net):
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(
        lambda p: p.requires_grad and len(p.data.size()) > 1, net.parameters()))))


if __name__ == "__main__":
    for net_name in __all__:
        if net_name.startswith('resnet'):
            print(net_name)
            test(globals()[net_name]())
            print()
    
archfunc = eval(arch + "()")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = archfunc.to(device)


if grayscale:
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
    summary(model, (1, 32, 32))
else:
    summary(model, (3, 32, 32))
    
best_prec1 = 0
best_prec5 = 0

# Training and Validation

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    '''
    Run one train epoch
    '''
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = input.cuda()
        target_var = target
        if half:
            input_var = input_var.half()

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 10)
        optimizer.step()

        output = output.float()
        loss = loss.float()

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}] | '
                  'Loss {loss.val:.4f} ({loss.avg:.4f}) | '
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f}) | '
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))
        
    return losses.val

In [ ]:
def validate(val_loader, model, criterion):
    '''
    Run evaluation
    '''
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()

            if half:
                input_var = input_var.half()

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))
            top5.update(prec5.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test\t  Prec@1: {top1.avg:.3f} (Err: {error1:.3f} )\n'
          'Test\t  Prec@5: {top5.avg:.3f} (Err: {error5:.3f} )'
          .format(top1=top1, error1=100-top1.avg, top5=top5, error5=100-top5.avg))

    return top1.avg, top5.avg


def save_checkpoint(state, filename='checkpoint.th'):
    '''
    Save the training model
    '''
    torch.save(state, filename)

In [ ]:
class AverageMeter(object):
    '''Computes and stores the average and current value'''

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class IterationCounter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.count = 0
        
    def update(self, val, n=1):
        self.count += n

def accuracy(output, target, topk=(1,)):
    '''Computes the precision@k for the specified values of k'''
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Dataset

In [ ]:
class LontarDataset(Dataset):
    def __init__(self,
                 annotations_file='../input/sundanese-lontar-character-dataset/train_labels.csv',
                 img_dir='../input/sundanese-lontar-character-dataset/train_image',
                 transform=None,
                 target_transform=None
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return (image, label)

## Data Balancing

In [ ]:
df = pd.read_csv('../input/sundanese-lontar-character-dataset/train_labels.csv')
labels = np.array(df.values.tolist())
classes = labels[:,1].astype(int)
class_sample_count = np.array(
    [
        len(np.where(labels == t)[0]) for t in np.unique(labels[:,1])
    ])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in classes])
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

## Functional Transforms

In [ ]:
class GaussBlur(object):
    def __init__(self, sigma=1):
        self.sigma = sigma
    
    def __call__(self, x):
        return TF.gaussian_blur(x, self.sigma)
    
class ThresholdTransform(object):
    def __init__(self, threshold):
        self.thr = threshold / 255.  # input threshold for [0..255] gray level, convert to [0..1]

    def __call__(self, x):
        return (x > self.thr).to(x.dtype)  # do not change the data type


## Dataset & Dataloaders

### Mean and Standard Deviation of the dataset
(kind of a quick and dirty solution)

In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        if grayscale:
            channels_sum += torch.mean(data)
            channels_squared_sum += torch.mean(data**2)
        else:
            channels_sum += torch.mean(data, dim=[0, 2, 3])
            channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
            
        num_batches += 1
        
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

transforms_norm_rgb = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor()
    ]

transforms_norm_rgb_ac = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0)
    ]

transforms_norm_rgb_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    GaussBlur()
    ]

transforms_norm_rgb_ac_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur()
    ]

transforms_norm_gray = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Grayscale()
    ]

transforms_norm_gray_ac = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.RandomAutocontrast(p=1.0)
    ]

transforms_norm_gray_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Grayscale(),
    GaussBlur()
    ]

transforms_norm_gray_ac_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur()
    ]

transforms_composes = [transforms_norm_rgb, transforms_norm_rgb_ac, transforms_norm_rgb_gauss, transforms_norm_rgb_ac_gauss, transforms_norm_gray, transforms_norm_gray_ac, transforms_norm_gray_gauss, transforms_norm_gray_ac_gauss]

for transforms_compose in transforms_composes:
    train_data_norm = LontarDataset(transform=transforms.Compose(transforms_compose))
    train_loader_norm = DataLoader(dataset=train_data_norm, batch_size=len(train_data_norm), num_workers=num_workers)
    dataset_mean, dataset_std = get_mean_and_std(train_loader_norm)
    print(f"mean={dataset_mean}, std={dataset_std}")

In [ ]:
transforms_compose = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    ]

transforms_compose_test = [
    transforms.Resize(resize),
    transforms.ToTensor(),
    ]

if do_normalize:
    transforms_compose.append(normalize)
    transforms_compose_test.append(normalize)
    
if grayscale:
    transforms_compose.append(transforms.Grayscale())
    transforms_compose_test.append(transforms.Grayscale())

print(transforms_compose)
    
train_data = LontarDataset(transform=transforms.Compose(transforms_compose))

test_data = datasets.ImageFolder(
    root='../input/sundanese-lontar-character-dataset/test_image',
    transform=transforms.Compose(transforms_compose_test))   

train_loader = DataLoader(dataset=train_data, batch_size=batch_size_train,
                          num_workers=num_workers, pin_memory=True, sampler=sampler)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size_val,
                         shuffle=False, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(dataset=test_data, batch_size=batch_size_val,
                        shuffle=True, num_workers=num_workers, pin_memory=True)

In [ ]:
transforms_rgb = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    ]

transforms_rgb_norm = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.7914, 0.6591, 0.4151], std=[0.1208, 0.1548, 0.1494])
    ]

transforms_rgb_ac = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0)
    ]

transforms_rgb_ac_normalize = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    transforms.Normalize(mean=[0.7431, 0.6549, 0.5137], std=[0.2067, 0.2260, 0.2094])
    ]
    
transforms_rgb_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    GaussBlur()
    ]

transforms_rgb_gauss_norm = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    GaussBlur(),
    transforms.Normalize(mean=[0.7914, 0.6591, 0.4151], std=[0.1208, 0.1548, 0.1494])
    ]
    
transforms_rgb_ac_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur()
    ]

transforms_rgb_ac_gauss_normalize = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur(),
    transforms.Normalize(mean=[0.7431, 0.6549, 0.5137], std=[0.2067, 0.2260, 0.2094])
    ]

################################################################

transforms_gray = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Grayscale()
    ]

transforms_gray_norm = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6708], std=[0.1418]),
    transforms.Grayscale()
    ]

transforms_gray_ac = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    transforms.Grayscale()
    ]

transforms_gray_ac_normalize = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    transforms.Normalize(mean=[0.6652], std=[0.2174]),
    transforms.Grayscale()
    ]
    
transforms_gray_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    GaussBlur(),
    transforms.Grayscale()
    ]

transforms_gray_gauss_norm = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    GaussBlur(),
    transforms.Normalize(mean=[0.6708], std=[0.1418]),
    transforms.Grayscale()
    ]
    
transforms_gray_ac_gauss = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur(),
    transforms.Grayscale()
    ]

transforms_gray_ac_gauss_normalize = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur(),
    transforms.Normalize(mean=[0.6652], std=[0.2174]),
    transforms.Grayscale()
    ]

transforms_gray_ac_binary = [
    transforms.ToPILImage(),
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.RandomAutocontrast(p=1.0),
    GaussBlur(),
    transforms.Normalize(mean=[0.6652], std=[0.2174]),
    transforms.Grayscale(),
    ThresholdTransform(threshold=169.6303609013557455)
    ]
    
transforms_composes_data = [
    transforms_rgb,
    transforms_rgb_norm,
    transforms_rgb_ac,
    transforms_rgb_ac_normalize,
    transforms_rgb_gauss,
    transforms_rgb_gauss_norm,
    transforms_rgb_ac_gauss,
    transforms_rgb_ac_gauss_normalize,
    transforms_gray,
    transforms_gray_norm,
    transforms_gray_ac,
    transforms_gray_ac_normalize,
    transforms_gray_gauss,
    transforms_gray_gauss_norm,
    transforms_gray_ac_gauss,
    transforms_gray_ac_gauss_normalize,
    transforms_gray_ac_binary
    ]

### Test Output

In [ ]:
# load classes from a txt file
classes_file = open(
    '../input/sundanese-lontar-character-dataset/list_class_name.txt', 'r')
classes = [(line.strip()).split() for line in classes_file]
classes_file.close()


# functions to show an image
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


def imshow(img):
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

for transforms_compose in transforms_composes_data:
    train_data = LontarDataset(transform=transforms.Compose(transforms_compose))
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size_train,
                          num_workers=num_workers, pin_memory=True, sampler=sampler)
    # get some random training images
    dataiter = iter(train_loader)
    images, labels = dataiter.next()
    plt.figure(figsize=(20, 10))

    # show images
    imshow(torchvision.utils.make_grid(images[0:8, :, :]))
    # print labels
    print(' '.join('%15s' % classes[labels[j]] for j in range(batch_size_train if batch_size_train < 8 else 10)))

# Run Experiment

In [ ]:
def main():
    global arch, half, weight_decay, momentum, lr, start_epoch, epochs, batch_size, evaluate, pretrained, print_freq, save_every, save_dir, best_prec1, best_prec5, avg_time, sum_time, optimizer, lr_scheduler, grayscale, do_normalize, normalize_rgb, normalize_gray
    
    data = []
    losses = []
    err1s = []
    err5s = []
    best_err1s = []
    best_err5s = []
    
    # Check the save_dir exists or not
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    model = archfunc
    model.cuda()

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    if half:
        print('half persicion is used.')
        model.half()
        criterion.half()

    optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                        milestones=[100, 150], last_epoch=start_epoch - 1)

    if evaluate:
        print('evalution mode')
        model.load_state_dict(torch.load(os.path.join(save_dir, 'model.th')))
        best_prec1, best_prec5 = validate(val_loader, model, criterion)
        return best_prec1, best_prec5

    if pretrained:
        print('evalution of pretrained model')
        save_dir = 'pretrained_models'
        pretrained_model = arch + '.th'
        model.load_state_dict(torch.load(
            os.path.join(save_dir, pretrained_model)))
        best_prec1, best_prec5 = validate(val_loader, model, criterion)
        return best_prec1, best_prec5

    # warmup before execution for time measurement
    
    #print("!!! WARMUP PHASE, DOES NOT REFLECT RESULT !!!")
    #train(train_loader, model, criterion, optimizer, 1)
    torch.cuda.synchronize()
    times = []

    # main training program
    for epoch in range(start_epoch, epochs):

        start_epoch = time.time()

        # train for one epoch
        print('Training {} model'.format(arch))
        print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
        data.append(train(train_loader, model, criterion, optimizer, epoch))
        losses.append(data[epoch])
        
        lr_scheduler.step()

        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        times.append(elapsed)

        # evaluate on validation set
        prec1, prec5 = validate(val_loader, model, criterion)
        err1 = 100-prec1
        err5 = 100-prec5

        err1s.append(err1)
        err5s.append(err5)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        best_prec5 = max(prec5, best_prec5)
        best_err1 = 100-prec1
        best_err5 = 100-prec5
        print('Current Best Prec @1: {best_prec1:.3f} (Err: {best_err1:.3f} ) | '
              '@5: {best_prec5:.3f} (Err: {best_err5:.3f} )\n'
              .format(epoch, best_prec1=best_prec1, best_err1=best_err1, best_prec5=best_prec5, best_err5=best_err5))
        best_err1s.append(100-best_prec1)
        best_err5s.append(100-best_prec5)

        if epoch > 0 and epoch % save_every == 0:
            save_checkpoint(model.state_dict(), filename=os.path.join(
                save_dir, 'checkpoint.th'))
        if is_best:
            save_checkpoint(model.state_dict(),
                            filename=os.path.join(save_dir, 'model.th'))

    model.load_state_dict(torch.load(os.path.join(save_dir, 'model.th')))
    best_prec1, best_prec5 = validate(val_loader, model, criterion)
    
    avg_time = sum(times)/epochs
    sum_time = sum(times)
    
    fig, (plt_loss, plt_error) = plt.subplots(1,2,figsize=(20,4))
    
    plt_loss.plot(losses, 'b-')
    plt_loss.set_xlabel('Epochs')
    plt_loss.set_ylabel('Loss')
    plt_loss.grid(True, 'major', 'both')
    
    plt_error.plot(err1s, label = "Error@1")
    plt_error.plot(err5s, label = "Error@5")
    plt_error.plot(best_err1s, label = "Best Error@1")
    plt_error.plot(best_err5s, label = "Best Error@5")
    plt_error.set_ylim([0,100])
    plt_error.set_xlabel('Epochs')
    plt_error.set_ylabel('Errors')
    plt_error.grid(True, 'major', 'both')
    
    plt.suptitle(arch)
    plt.legend(loc=1)
    plt.show()
    
    return best_prec1, best_prec5

In [ ]:
if __name__ == '__main__':
    current_device = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(0)
    
    best_prec1, best_prec5 = main()
    
    print(device_name)
    print('Training took {} seconds (avg. {} per epoch).\n'
          'The training accuracy from {} model after {} epochs is: \n'
          'Prec@1 = {acc1:.3f}\n'
          'Prec@5 = {acc5:.3f}'.format(
           sum_time, avg_time, arch, epochs, acc1 = best_prec1, acc5 = best_prec5))

In [ ]:
import IPython.display as ipd

audio_url = "https://assets.mixkit.co/sfx/download/mixkit-happy-bells-notification-937.wav"
ipd.Audio(audio_url, autoplay=True)